In [0]:
CREATE DATABASE sales_scd;

In [0]:
CREATE OR REPLACE TABLE sales_scd.Orders (
    OrderID INT PRIMARY KEY,
    OrderDate DATE,
    CustomerID INT,
    CustomerName VARCHAR(100),
    CustomerEmail VARCHAR(100),
    ProductID INT,
    ProductName VARCHAR(100),
    ProductCategory VARCHAR(50),
    RegionID INT,
    RegionName VARCHAR(50),
    Country VARCHAR(50),
    Quantity INT,
    UnitPrice DECIMAL(10,2),
    TotalAmount DECIMAL(10,2)
);


In [0]:



INSERT INTO sales_scd.Orders (
    OrderID, OrderDate, CustomerID, CustomerName, CustomerEmail,
    ProductID, ProductName, ProductCategory,
    RegionID, RegionName, Country,
    Quantity, UnitPrice, TotalAmount
)
VALUES
(1, '2024-02-01', 101, 'Alice Johnson', 'alice@example.com',
 201, 'Laptop', 'Electronics',
 301, 'North America', 'USA',
 2, 800.00, 1600.00),

(2, '2024-02-02', 102, 'Bob Smith', 'bob@example.com',
 202, 'Smartphone', 'Electronics',
 302, 'Europe', 'Germany',
 1, 500.00, 500.00),

(3, '2024-02-03', 103, 'Charlie Brown', 'charlie@example.com',
 203, 'Tablet', 'Electronics',
 303, 'Asia', 'India',
 3, 300.00, 900.00),

(4, '2024-02-04', 101, 'Alice Johnson', 'alice@example.com',
 204, 'Headphones', 'Accessories',
 301, 'North America', 'USA',
 1, 150.00, 150.00),

(5, '2024-02-05', 104, 'David Lee', 'david@example.com',
 205, 'Gaming Console', 'Electronics',
 302, 'Europe', 'France',
 1, 400.00, 400.00),

(6, '2024-02-06', 102, 'Bob Smith', 'bob@example.com',
 206, 'Smartwatch', 'Electronics',
 303, 'Asia', 'Japan',
 2, 250.00, 500.00),

(7, '2024-02-07', 105, 'Emma Wilson', 'emma@example.com',
 207, 'Keyboard', 'Accessories',
 301, 'North America', 'Canada',
 2, 120.00, 240.00),

(8, '2024-02-08', 106, 'Frank Miller', 'frank@example.com',
 208, 'Monitor', 'Electronics',
 302, 'Europe', 'UK',
 1, 300.00, 300.00),

(9, '2024-02-09', 107, 'Grace Taylor', 'grace@example.com',
 209, 'Mouse', 'Accessories',
 303, 'Asia', 'Singapore',
 3, 80.00, 240.00),

(10, '2024-02-10', 108, 'Henry Adams', 'henry@example.com',
 210, 'Printer', 'Electronics',
 301, 'North America', 'USA',
 1, 350.00, 350.00);






num_affected_rows,num_inserted_rows
10,10


### SCD TYPE 1

In [0]:
SELECT * FROM sales_scd.Orders

OrderID,OrderDate,CustomerID,CustomerName,CustomerEmail,ProductID,ProductName,ProductCategory,RegionID,RegionName,Country,Quantity,UnitPrice,TotalAmount
1,2024-02-01,101,Alice Johnson,alice@example.com,201,Laptop,Electronics,301,North America,USA,2,800.00,1600.00
2,2024-02-02,102,Bob Smith,bob@example.com,202,Smartphone,Electronics,302,Europe,Germany,1,500.00,500.00
3,2024-02-03,103,Charlie Brown,charlie@example.com,203,Tablet,Electronics,303,Asia,India,3,300.00,900.00
4,2024-02-04,101,Alice Johnson,alice@example.com,204,Headphones,Accessories,301,North America,USA,1,150.00,150.00
5,2024-02-05,104,David Lee,david@example.com,205,Gaming Console,Electronics,302,Europe,France,1,400.00,400.00
6,2024-02-06,102,Bob Smith,bob@example.com,206,Smartwatch,Electronics,303,Asia,Japan,2,250.00,500.00
7,2024-02-07,105,Emma Wilson,emma@example.com,207,Keyboard,Accessories,301,North America,Canada,2,120.00,240.00
8,2024-02-08,106,Frank Miller,frank@example.com,208,Monitor,Electronics,302,Europe,UK,1,300.00,300.00
9,2024-02-09,107,Grace Taylor,grace@example.com,209,Mouse,Accessories,303,Asia,Singapore,3,80.00,240.00
10,2024-02-10,108,Henry Adams,henry@example.com,210,Printer,Electronics,301,North America,USA,1,350.00,350.00


### IN ORDER TO APPLY SLOWLY CHANGING DIMENSION WE FIRST NEED TO CREATE A VIEW AND ON TOP OF EXISTING DIMENSION

In [0]:
CREATE OR REPLACE VIEW sales_scd.viewDim_Products AS

SELECT DISTINCT(ProductID) as ProductID, ProductName, ProductCategory FROM sales_scd.Orders
WHERE OrderDate > '2024-02-10'

In [0]:
CREATE OR REPLACE TABLE sales_scd.DimProducts(
  ProductID INT PRIMARY KEY,
  ProductName STRING,
  ProductCategory STRING
  





)

In [0]:
INSERT INTO sales_scd.DimProducts

SELECT ProductID, ProductName, ProductCategory FROM sales_scd.viewDim_Products


num_affected_rows,num_inserted_rows
10,10


In [0]:
SELECT * FROM sales_scd.DimProducts

ProductID,ProductName,ProductCategory
204,Headphones,Accessories
205,Gaming Console,Electronics
207,Keyboard,Accessories
209,Mouse,Accessories
202,Smartphone,Electronics
206,Smartwatch,Electronics
201,Laptop,Electronics
203,Tablet,Electronics
208,Monitor,Electronics
210,Printer,Electronics


### ON NEXT DAY WE HAVE CHANGING IN PRODUCT ID OF 210. NOW WE HAVE "LASER JET PRINTER INSTEAD OF PRINTER". SCD SHOULD UPDATE INFO  IN DIM TABLE and WE HAVE A NEW RECORD with productID 211, product name is airpods and product category electronics 

In [0]:
INSERT INTO sales_scd.Orders (
    OrderID, OrderDate, CustomerID, CustomerName, CustomerEmail,
    ProductID, ProductName, ProductCategory,
    RegionID, RegionName, Country,
    Quantity, UnitPrice, TotalAmount
)
VALUES 

(10, '2024-02-11', 108, 'Henry Adams', 'henry@example.com',
 210, ' LASER JET Printer', 'Electronics',
 301, 'North America', 'USA',
 1, 350.00, 350.00),

 (11, '2024-02-12', 211, 'Alice Johnson', 'alice@example.com',
 211, 'AIR PODS', 'Electronics',
 301, 'North America', 'USA',
 2, 800.00, 1600.00);




num_affected_rows,num_inserted_rows
2,2


### CHANGING VIEW AFTER ADDING NEW DATA 

In [0]:
SELECT * FROM sales_scd.viewDim_Products

ProductID,ProductName,ProductCategory
210,LASER JET Printer,Electronics
211,AIR PODS,Electronics


### MERGE SCD- TYPE 1

In [0]:
MERGE INTO sales_scd.DimProducts AS target
USING sales_scd.viewDim_Products AS source
ON target.ProductID = source.ProductID
WHEN MATCHED THEN UPDATE SET *
WHEN NOT MATCHED THEN INSERT *

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
2,1,0,1


In [0]:
SELECT * FROM sales_scd.DimProducts

ProductID,ProductName,ProductCategory
204,Headphones,Accessories
205,Gaming Console,Electronics
207,Keyboard,Accessories
209,Mouse,Accessories
202,Smartphone,Electronics
206,Smartwatch,Electronics
201,Laptop,Electronics
203,Tablet,Electronics
208,Monitor,Electronics
211,AIR PODS,Electronics
